### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 5 sobre las plantas medicinales creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga una página principal que mostrará un conjunto enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * __Servicio 1__: Mostrar información sobre una planta seleccionada. Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que dispondrá varios desplegables donde podrá seleccionar la planta. Un seleccionable para elegir el grupo , y otro para seleccionar la planta del grupo que ha elegido.  Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página que mostrará la descripción de la planta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]
   
   
   * __Servicio 2__:Buscar planta por palabra clave. Si elige esta opción se le pedirá al usuario que introduzca un conjunto de palabras que se utilizarán para realizar una búsqueda sobre el texto de las descripciones asociadas a las plantas que aparecen en cada página. A continuación, si ha introducido más de una palabra, se le preguntará si quiere realizar una búsqueda de tipo "AND" o una búsqueda de tipo "OR", es decir si busca plantas donde aparece todas las palabras introducidas o si busca plantas donde aparecen alguna de las palabras introducidas. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con un listado de todas las plantas junto a las descripciones de las mismas que cumplen las condiciones de búsqueda.  En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

   * __Servicio 3__:Buscar plantas por enfermedades. Si elige esta opción se le mostrará un listado chequeable donde seleccionará una o más enfermedades. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con todas las plantas que pueden venir bien para esa enfermedad. Para ello se proporciona junto a la práctica un csv con nombres de enfermedades que servirá de entrada para generar el listado de enfermedades.En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

## Normas de entrega

* Fecha tope de entrega: 31/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [5]:
# Practica 5

import urllib
import re
from bs4 import BeautifulSoup
hrefBreak = ["Notas y referencias","Referencias", "Ver también"]
hrefContinue = ["Usos en medicina herbolaria", "Usos en la alimentación"]


#Estructura de datos para almacenar la informacion de las paginas web en forma de diccionario
#Los pares clave-valor consisten en la web y como valor para cada web otro diccionario formado por 
#los pares clave-valor que son nombre de la planta y como valor una tupla [nombreCientifico, informacion]
#Acceso a lista de webs --> list.(diccionarioPlantas.keys())
#Acceso a lista de plantas de una web --> list.(diccionarioPlantas[web].keys())
#Acceso a toda la informacion de una planta --> diccionarioPlantas[web][nombrePlanta]
#Acceso a nombre cientifico de una planta --> diccionarioPlantas[web][nombrePlanta][0]
#Acceso a informacion de una planta --> diccionarioPlantas[web][nombrePlanta][1]

diccionarioPlantas={"https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(A-B)":{},
        "https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(C)":{},
        "https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(D-G)":{},
        "https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(H-M)":{},
        "https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(N-Z)":{}}


"""Funcion que añade el nombre de las plantas"""
def devuelvePlantas(url):
    html=urllib.request.urlopen(url).read()
    soup=BeautifulSoup(html, 'html.parser')
    for planta in soup.ul.find_all('a'):
        nombre = planta.get("href",None).lstrip("#").replace("_"," ")
        if nombre in hrefBreak:
            break;
        elif nombre in hrefContinue:
            continue;
        else:
            diccionarioPlantas[url][nombre] = ["",""]

      
    
def limpiaTexto(texto):
    nombreCientifico = ""
    informacion = ""
    for line in texto.split('\n'):
        if (nombreCientifico == ""):
            nombreCientifico = line
        else:
            informacion += line
    informacion = re.sub("\[(.*?)\]", "", informacion)
    informacion = re.sub("[\\u200b]","",informacion)
    informacion = informacion.replace("\\","")
    return [nombreCientifico,informacion]


def extraeInformacionPlanta(planta, web, indice):
    seguir = True
    texto = ""
    while(seguir):
        planta = planta.next_sibling
        if hasattr(planta, 'name') and (planta.name == "h3" or planta.name == "h2"):
            seguir = False
            datos = limpiaTexto(texto)
            diccionarioPlantas[web][list(diccionarioPlantas[web].keys())[indice]][0] = datos[0]
            diccionarioPlantas[web][list(diccionarioPlantas[web].keys())[indice]][1] = datos[1]                
        elif (hasattr(planta, 'text') and planta.name != "div" and planta.name != "h2"):
                texto += planta.text + "\n"
                
                
"""Funcion que genera un diccionario con todos los datos requeridos"""
def sacarInformacionPlantas():
    for web in diccionarioPlantas:
        html=urllib.request.urlopen(web).read()
        soup=BeautifulSoup(html, 'html.parser')
        #Se itera sobre los elementos h3
        i = 0
        for cabecera in soup.find_all("h3"):
            #Variable boolean para marcar cuando se encuentra otro h3
            extraeInformacionPlanta(cabecera, web, i)
            i += 1
            if i == len(diccionarioPlantas[web].keys()):
                break



def menu():
    opcion = -1
    print("\n------------------- MENU -------------------")
    print("1. Buscar por índice")
    print("2. Buscar por palabra clave")
    print("0. Terminar ejecucion ")
    print("--------------------------------------------\n")
    
    opcion = input("Elige una opción: ")
    if opcion.isdigit():
        opcion = int(opcion)
        if(opcion >= 0 and opcion <= 2):
            return int(opcion)
    print("Opción no válida\n")
    return -1

def menuPorIndice():
    opcion = -1
    print("------------------- CRITERIO -------------------")
    i = 1
    for web in diccionarioPlantas:
        print(i, ". ", web.split("Anexo:")[1].replace("_", " "))
        i+=1
    print("0 . ", "Salir")
    print("--------------------------------------------\n")

    opcion = input("Elige una opción: ")
    if opcion.isdigit():
        opcion = int(opcion)
        if(opcion >= 0 and opcion <= 5):
            return opcion
    print("Opción no válida\n")
    return -1
    
    
def menuSeleccionPlanta(plantas):
    print("Seleccion de planta (",len(plantas),") Resultados" )
    i = 1
    for planta in plantas:
        print(i, ".", planta)
        i+=1
    print("0 . ", "Salir")
    listaPlantas=list(plantas)
    opcion = input("\nElige una planta:\n")
    if opcion.isdigit():
        opcion = int(opcion)
        if(opcion > 0 and opcion <= len(plantas)): #Mostrar datos de la planta
            planta=listaPlantas[opcion-1]
            return planta
        else:
            return None


def menuORAND():
    opcion = -1
    print("\n------------ TIPO DE BÚSQUEDA ------------")
    print("1. Búsqueda AND")
    print("2. Búsqueda OR")
    print("0. Salir")
    print("--------------------------------------------\n")
    
    opcion = input("Elige una opción: ")
    if opcion.isdigit():
        opcion = int(opcion)
        if(opcion >= 1 and opcion <= 2):
            return opcion
    print("Opción no válida\n")
    return -1

def escribePlanta(planta, informacion):
    print("\033[1mPlanta: \033[0m", str(planta))
    print("\033[1mNombre cientifico: \033[0m",informacion[0])
    print("\033[1mInformacion: \033[0m",informacion[1], "\n")

#------------------------OR-------------
def buscaOrEnPlanta(informacion, listaPalabras):
    for palabra in listaPalabras:
        r = re.compile(r'\b%s\b' % palabra, re.I)
        if r.search(informacion):
            return True
    return False

def busquedaOR(listaPalabras):
    print("Busqueda OR\n")
    for url in diccionarioPlantas.keys():
        for planta in diccionarioPlantas[url]:
            if (buscaOrEnPlanta(diccionarioPlantas[url][planta][1],listaPalabras)):
                escribePlanta(planta, diccionarioPlantas[url][planta])

                
#------------------------AND---------------
def buscaAndEnPlanta(informacion, listaPalabras):
    for palabra in listaPalabras:
        r = re.compile(r'\b%s\b' % palabra, re.I)
        if not r.search(informacion):
            return False
    return True

def busquedaAND(listaPalabras):
    print("Busqueda AND\n")
    for url in diccionarioPlantas.keys():
        for planta in diccionarioPlantas[url]:
            if (buscaAndEnPlanta(diccionarioPlantas[url][planta][1],listaPalabras)):
                escribePlanta(planta, diccionarioPlantas[url][planta])
    

def preguntarNuevaBusqueda():
    opcion = input("Quiere realizar una nueva búsqueda (S/N): ")
    opcion = opcion.lower()
    if opcion == "s":
        return True
    else:
        return False

    
    
def busquedaPorIndice():
    print("\nBúsqueda por índice\n")
    opcion = menuPorIndice()
    if(opcion != -1):
        if(opcion != 0):
            url = list(diccionarioPlantas.keys())[opcion-1]
            planta = menuSeleccionPlanta(diccionarioPlantas[url])
            if planta != None:
                escribePlanta(planta, diccionarioPlantas[url][planta])
            else:
                print("Indice no valido")
    
def busquedaPorPalabra():
    print("\nBúsqueda por palabra clave\n")
    palabras = []
    palabra = None
    while(palabra != "0"):
        palabra = input("Escribe la palabra de búsqueda: (0 para terminar)\n")
        if(palabra != "0" and palabra != ""):
            palabras.append(palabra)
        elif palabra == "0":
            break
    if(len(palabras) == 0):
        print("Error, minimo una palabra")
    elif(len(palabras) == 1):
        print("Buscando la palabra", palabras[0])
        encontrado = False
        busquedaOR(palabras)
    else:
        print("Buscando las palabras", palabras)
        #Pedir OR o AND
        seguir = True
        opcion = menuORAND()
        if(opcion == 1):
            # Busqueda AND
            busquedaAND(palabras)
        elif(opcion == 2):
            # Búsqueda OR
            busquedaOR(palabras)
    
#-------------------------------------------------------------
def buscador():
    opcion = -1
    buscar = True
    for url in diccionarioPlantas:
        devuelvePlantas(url)
    sacarInformacionPlantas()

    while buscar:
        opcion = menu()
        if (opcion == 0):
            buscar = False
        elif(opcion == 1):
            busquedaPorIndice()
            buscar = preguntarNuevaBusqueda()
        elif (opcion == 2):
            busquedaPorPalabra()
            buscar = preguntarNuevaBusqueda()


    
buscador()
print("Terminando la ejecución...")




------------------- MENU -------------------
1. Buscar por índice
2. Buscar por palabra clave
0. Terminar ejecucion 
--------------------------------------------

Elige una opción: 
Opción no válida


------------------- MENU -------------------
1. Buscar por índice
2. Buscar por palabra clave
0. Terminar ejecucion 
--------------------------------------------

Elige una opción: 1

Búsqueda por índice

------------------- CRITERIO -------------------
1 .  Plantas medicinales (A-B)
2 .  Plantas medicinales (C)
3 .  Plantas medicinales (D-G)
4 .  Plantas medicinales (H-M)
5 .  Plantas medicinales (N-Z)
0 .  Salir
--------------------------------------------

Elige una opción: 1
Seleccion de planta ( 20 ) Resultados
1 . Albahaca
2 . Aceituno
3 . Agastache
4 . Ajenjo
5 . Alcotán
6 . Manzanilla
7 . Amargón
8 . Apacin
9 . Epazote
10 . Apio
11 . Arrayán
12 . Árnica
13 . Bacche
14 . Barbasco
15 . Bolsa de pastor
16 . Berro
17 . Boldo
18 . Borraja
19 . Bretónica
20 . Buganvilea
0 .  Salir

Eli

Quiere realizar una nueva búsqueda (S/N): s

------------------- MENU -------------------
1. Buscar por índice
2. Buscar por palabra clave
0. Terminar ejecucion 
--------------------------------------------

Elige una opción: 2

Búsqueda por palabra clave

Escribe la palabra de búsqueda: (0 para terminar)
vitamina
Escribe la palabra de búsqueda: (0 para terminar)
0
Buscando la palabra vitamina
Busqueda OR

Planta:  Arrayán
Nombre cientifico:  (Eugenia allí y otras especies)
Informacion:  Son utilizables hojas y fruta.Se utiliza por sus propiedades astringentes, antisépticas, balsámicas (para curar heridas, llagas u otras enfermedades), y hemostáticas (para controlar hemorragias), en infusiones, decocciones y productos cosméticos.En la medicina popular, el arrayán ha sido utilizado para el tratamiento de la diabetes. Se ha comprobado que extractos acuosos de arrayán tienen efectos hipoglicemiantes (disminuye el azúcar en la sangre, sin actuar sobre los niveles de glucosa normales. Se pr

Quiere realizar una nueva búsqueda (S/N): n
Terminando la ejecución...


In [ ]:
from bottle import route, run, get, post, template

# En el navegador -> http://localhost:8080/index

# Para probar que funciona
@route ('/hola')
def hola():
    return "<h1> ¡Hola Mundo!</h1>"

# Parece que hay que poner \ al final de cada linea para que escape el enter y lo coja bien
# si no da un error de indentacion. Esto por ejemplo valdria como formulario para seleccionar
# algo de un menu
@route ('/select')
def seleccion():
    return '<form action="/<aqui ira la funcion a la que ir>" method="post">\
                Seleccion: <input type="text" />\
                <input value="seleccion" type="submit" />\
            /form>"'

# Solamente esta como estructura no hay nada pero para ir probando
@route ('/index')
def index():
    return template('./web/index.tpl')
                    

run(host='0.0.0.0', port=8080)